In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.llms.openai import OpenAI

chat = OpenAI(
    temperature=0.1,
    max_tokens=450,
    model="gpt-3.5-turbo-16k",
)

chat.save("model.json")

Arrr matey! Me favorite grub be a hearty plate of salted fish and hardtack! Aye, nothing beats the taste of the sea on me tongue! Arrr!

AIMessageChunk(content='Arrr matey! Me favorite grub be a hearty plate of salted fish and hardtack! Aye, nothing beats the taste of the sea on me tongue! Arrr!')